# CityBikes

Send a request to CityBikes for the city of your choice. 

In [3]:
# import necessary libraries
import requests
import pandas as pd
import json


In [ ]:
# make a pull request on the networks
response = requests.get("https://api.citybik.es/v2/networks")
results_dict = response.json()

# explore json file structure
results_dict.keys()

In [ ]:
# see inside networks dictionary
results_dict["networks"]
response_network = results_dict['networks']
response_network[0].keys()

In [ ]:
# Get view of location dictionary structure
response_network[0]['location']

In [ ]:
# Make a function to pull for Tokyo
def tokyo_networks(response_network):
    network_list = []
    for i in range(len(response_network)):
        if 'tokyo' in response_network[i]['location']['city'].lower():
            network_list.append(response_network[i])
    return network_list

values_tokyo = tokyo_networks(response_network)


In [ ]:
# Output shows multiple networks. Will pick docomo-cycle-tokyo network for endpoint API
tokyo_docomo = requests.get(values_tokyo[0]['gbfs_href'])
tokyo_json = tokyo_docomo.json()
tokyo_json['data']
# Find location of station information and make a new pull
station = tokyo_json['data']['ja']['feeds'][2]
station_info = requests.get(station['url'])
station_info = station_info.json()
station_info

Parse through the response to get the details you want for the bike stations in that city (latitude, longitude, number of bikes). 

In [ ]:
# Go through docomo tokyo network station_info response for location information
station_location = station_info['data']['stations']
station_location

Put your parsed results into a DataFrame.

In [13]:
# Save station_location json file
with open('tokyo_bike_stations.json', 'w') as f:
    json.dump(station_location, f)

In [5]:
# Store created station json file into a variable to avoid making constant calls
with open('tokyo_bike_stations.json') as f:
    stations_data = json.load(f)


In [11]:
# store data in dataframe and add station column 
df_stations = pd.DataFrame.from_records(stations_data)
df_stations
station_number = []
for i in range(len(stations_data)):
    station_number.append(i + 1) 

df_stations['station_number'] = station_number
df_stations

,lat,lon,name,capacity,region_id,station_id,station_number
0,35.693799,139.753281,A1-01.千代田区役所,5,1,00010001,1
1,35.701073,139.747525,A1-03.東京区政会館（メトロA5出口）,7,1,00010003,2
2,35.701347,139.746951,A1-04.東京区政会館（西側駐輪場）,6,1,00010004,3
3,35.699187,139.746348,A1-05.富士見出張所,9,1,00010005,4
4,35.699132,139.744620,A1-08.飯田橋サクラテラス（早稲田通り）,10,1,00010036,5
...,...,...,...,...,...,...,...
1577,35.514643,139.706285,Z21.コーナン川崎小田栄店,5,11,00000205,1578
1578,35.532615,139.700332,Z22.みんちゅう京急川崎駅前駐輪場内ラックスペース,3,11,00000928,1579
1579,35.524976,139.697495,Z23.NA川崎南町,5,11,00001583,1580
1580,35.525854,139.717848,Z24.総合川崎臨港病院,4,11,00002196,1581


In [ ]:
# store Values to be used for data joining 
%store stations_data
%store df_stations

Stored 'stations_data' (list)
Stored 'df_stations' (DataFrame)
